In [1]:
import mysql.connector as con
import pandas as pd
import random
import datetime 
from datetime import datetime, timedelta


## ENUNCIADO EJERCICIO 

### 1. CREAR BASE DE DATOS (33%)

* Crear con código SQL una base de datos llamada supermercado.
* Código Python con mysql connector que ejecute el SQL para borrar y generar la base de datos vacía.

In [2]:
def crear_tabla(sql):
    try:
        connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="admin",
            database='supermercado'
        )
        cursor = connection.cursor()
        cursor.execute(sql)
    except con.Error as error:
        print(f"Ha ocurrido un error: {error}")
        if connection:
            connection.rollback()
        return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

In [3]:
sql = "DROP DATABASE IF EXISTS supermercado; CREATE DATABASE IF NOT EXISTS supermercado;"

In [4]:
crear_tabla(sql)

## Crear TABLAS

tiendas
- id_tienda (PRIMARY KEY)
- nombre_tienda
- direccion
- ciudad

In [5]:
sql = "CREATE TABLE IF NOT EXISTS tiendas (id_tienda INT AUTO_INCREMENT PRIMARY KEY, nombre_tienda VARCHAR(255) NOT NULL, direccion VARCHAR(255), ciudad VARCHAR(255) NOT NULL);"

In [6]:
crear_tabla(sql)

empleados
- id_empleado (PRIMARY KEY)
- nombre_empleado
- puesto (ej.: Cajero, Gerente, Reponedor)
- id_tienda (FOREIGN KEY que hace referencia a tiendas.id_tienda)

In [7]:
sql = "CREATE TABLE IF NOT EXISTS empleados (id_empleado INT AUTO_INCREMENT PRIMARY KEY, nombre_empleado VARCHAR(255) NOT NULL, puesto VARCHAR(60), id_tienda INT NOT NULL, FOREIGN KEY (id_tienda) REFERENCES tiendas(id_tienda) ON DELETE CASCADE);"

In [8]:
crear_tabla(sql)

categorias
- id_categoria (PRIMARY KEY)
- nombre_categoria

In [9]:
sql = "CREATE TABLE IF NOT EXISTS categorias (id_categoria INT AUTO_INCREMENT PRIMARY KEY, nombre_categoria VARCHAR(60));"

In [10]:
crear_tabla(sql)

productos
- id_producto (PRIMARY KEY)
- nombre_producto
- precio
- stock
- id_categoria (FOREIGN KEY que hace referencia a categorias.id_categoria)

In [11]:
sql = "CREATE TABLE IF NOT EXISTS productos (id_producto INT AUTO_INCREMENT PRIMARY KEY, nombre_producto VARCHAR(70), precio FLOAT(8,2), stock INT, id_categoria INT NOT NULL, FOREIGN KEY (id_categoria) REFERENCES categorias(id_categoria));"

In [12]:
crear_tabla(sql)

clientes
- id_cliente (PRIMARY KEY)
- first_name
- last_name
- email
- codigo_postal

In [13]:
sql = "CREATE TABLE IF NOT EXISTS clientes (id_cliente INT AUTO_INCREMENT PRIMARY KEY, first_name VARCHAR(50), last_name VARCHAR(50), email VARCHAR(200), codigo_postal INT);"

In [14]:
crear_tabla(sql)

ordenes
- id_orden (PRIMARY KEY)
- id_cliente (FOREIGN KEY que hace referencia a clientes.id_cliente)
- id_empleado (FOREIGN KEY que hace referencia a empleados.id_empleado)
- fecha_orden
- metodo_pago (una enum que solo admita tres valores Tarjeta, Efectivo)

In [15]:
sql = "CREATE TABLE IF NOT EXISTS ordenes (id_orden INT AUTO_INCREMENT PRIMARY KEY, id_cliente INT, id_empleado INT, fecha_orden DATE, metodo_pago ENUM('Tarjeta', 'Efectivo') DEFAULT 'Efectivo', FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente), FOREIGN KEY (id_empleado) REFERENCES empleados(id_empleado));"

In [16]:
crear_tabla(sql)

detalle_orden
- id_detalle (PRIMARY KEY)
- id_orden (FOREIGN KEY que hace referencia a ordenes.id_orden) NOT NULL
- id_producto (FOREIGN KEY que hace referencia a productos.id_producto) NOT NULL
- cantidad
- precio_unitario: mismo precio que en la tabla producto
- descuento (podría ser NULL si no se aplica)

In [17]:
sql = "CREATE TABLE IF NOT EXISTS detalle_orden (id_detalle INT AUTO_INCREMENT PRIMARY KEY, id_orden INT NOT NULL, id_producto INT NOT NULL, cantidad FLOAT(8,2), precio_unitario FLOAT(8,2), descuento TINYINT, FOREIGN KEY (id_orden) REFERENCES ordenes(id_orden), FOREIGN KEY (id_producto) REFERENCES productos(id_producto));"

In [18]:
crear_tabla(sql)

### 2. Generar datos demo desde Python (33%)

* Generar datos aleatorios en listas con Python similar los realizados en clase.
    * Uso de datetime, timedelta, random para generar datos aleatorios
* Pasar los datos a DataFrames de Pandas
* Pasar los DataFrames de Pandas a MySQL usando la función to_sql de Pandas con SQLAlchemy o usando MySQL Connector con sentencias INSERT.

In [19]:
def insertar_dataframe(df, table_name):
    try:
        connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="admin",
            database='supermercado'
        )
        cursor = connection.cursor()
        
        columns = ','.join(df.columns)
        placeholders = ','.join(['%s'] * len(df.columns))
        sql = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders});'
    
        rows = [tuple(row) for index, row in df.iterrows()] # lista de tuplas con los datos del dataframe

        cursor.executemany(sql, rows)
        
        connection.commit()
        return cursor.rowcount
    except con.Error as error:
        print(f"Ha ocurrido un error: {error}")
        if connection:
            connection.rollback()
        return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

tiendas

* id_tienda: valores enteros consecutivos (por ejemplo, 1, 2, 3...).
* nombre_tienda: nombres ficticios o genéricos (p. ej. "Tienda Centro", "Super Norte", "Super Sur").
* direccion: direcciones simples (p. ej. “Calle Falsa 123”).
* ciudad: usar nombres de ciudades ficticias o reales (p. ej. “Madrid”, “Barcelona”, “México DF”, etc.).

En total 5 o 10 tiendas.

In [20]:
tiendas = ['Jerez Este', 'Sanlucar Oeste', 'Cadiz Norte', 'Valdelagrana Sur', 'Arcos Sur', 'Chiclana Norte', 'Barbate Este', 'Vejer Oeste', 'Conil Sur', 'Rota Norte']
direcciones = ['Calle Catavino 25', 'Calle Manzanilla 42', 'Calle del Pan 38', 'Calle Playa 57', 'Calle Castillo 182', 'Calle Barrosa 63', 'Calle Atun 98', 'Calle Pescadill 56', 'Calle Balsa 71', 'Calle Base 22']
ciudades = ['Jerez de la Frontera', 'Sanlucar de Barrameda', 'Cadiz', 'El Puerto de Santa Maria', 'Arcos de la Frontera', 'Chiclana de la Frontera', 'Barbate', 'Vejer de la Frontera', 'Conil de la Frontera', 'Rota']
df_tiendas = pd.DataFrame(zip(tiendas, direcciones, ciudades), columns=['nombre_tienda', 'direccion', 'ciudad'])
# generar columna id_tienda
df_tiendas = df_tiendas.reset_index().rename({'index': 'id_tienda'}, axis=1)
df_tiendas['id_tienda'] = df_tiendas['id_tienda'] + 1
df_tiendas

,id_tienda,nombre_tienda,direccion,ciudad
0,1,Jerez Este,Calle Catavino 25,Jerez de la Frontera
1,2,Sanlucar Oeste,Calle Manzanilla 42,Sanlucar de Barrameda
2,3,Cadiz Norte,Calle del Pan 38,Cadiz
3,4,Valdelagrana Sur,Calle Playa 57,El Puerto de Santa Maria
4,5,Arcos Sur,Calle Castillo 182,Arcos de la Frontera
5,6,Chiclana Norte,Calle Barrosa 63,Chiclana de la Frontera
6,7,Barbate Este,Calle Atun 98,Barbate
7,8,Vejer Oeste,Calle Pescadill 56,Vejer de la Frontera
8,9,Conil Sur,Calle Balsa 71,Conil de la Frontera
9,10,Rota Norte,Calle Base 22,Rota


In [21]:
insertar_dataframe(df_tiendas, 'tiendas')

10

empleados

* id_empleado: valores enteros consecutivos (1, 2, 3...).
* nombre_empleado: nombres y apellidos ficticios (p. ej. “Laura Gutiérrez”, “Juan Pérez”).
* puesto: limitarse a un conjunto reducido de valores (p. ej. {‘Cajero’, ‘Gerente’, ‘Reponedor’, ‘Vendedor’}).
* id_tienda: debe hacer referencia a una tienda existente (por ejemplo, un número entre 1 y 3 si solo tienes 3 tiendas).

En total 20 empleados por tienda.

In [22]:
#generar columna id_empleado
empleados = list()
nombres = ["Juan", "María", "Luis", "Ana", "Pedro", "Sofía", "Carlos", "Lucía", "Jorge", "Elena", "Manuel", "Laura", "Diego", "Clara", "Alberto", "Teresa", "Pablo", "Isabel", "Sergio", "Marta", "Roberto", "Julia", "Fernando", "Carmen", "Ricardo", "Paula", "Andrés", "Eva", "Antonio", "Sara", "José", "Natalia", "Francisco", "Patricia", "Hugo", "Adriana", "Rafael", "Irene", "Víctor", "Olga", "Daniel", "Noelia", "Álvaro", "Alicia", "Javier", "Rocío", "Enrique", "Amelia", "Gonzalo", "Vera"]
apellidos = ["García", "Martínez", "López", "Sánchez", "Pérez", "González", "Rodríguez", "Fernández", "Moreno", "Jiménez", "Hernández", "Muñoz", "Álvarez", "Romero", "Alonso", "Gutiérrez", "Ruiz", "Navarro", "Díaz", "Torres", "Castro", "Vázquez", "Ramos", "Gil", "Ramírez", "Flores", "Ortiz", "Marín", "Serrano", "Iglesias", "Delgado", "Domínguez", "Guerrero", "Molina", "Ortega", "Rubio", "Santos", "Medina", "Castillo", "Cruz", "Reyes", "Herrera", "Peña", "Carrasco", "Calvo", "Vega", "Cabrera", "Campos", "León", "Fuentes"]
trabajo = ["Cajero", "Reponedor", "Encargado de sección", "Gerente", "Personal de limpieza", "Panadero", "Carnicero", "Pescadero", "Responsable de almacén", "Supervisor de caja", "Frutero", "Charcutero", "Jefe de turno", "Operador logístico", "Atención al cliente", "Auxiliar de caja", "Encargado de inventario", "Responsable de compras", "Promotor de ventas", "Encargado de seguridad"]
for tienda in df_tiendas['id_tienda']:
    for empleado in range(20):
        nombre_empleado = (random.choice(nombres) + ' ' + random.choice(apellidos))
        puesto = trabajo[empleado]
        id_tienda = tienda
        empleados.append([nombre_empleado, puesto, id_tienda])
        
df_empleados = pd.DataFrame(empleados, columns=['nombre_empleado', 'puesto', 'id_tienda'])
        
df_empleados = df_empleados.reset_index().rename({'index': 'id_empleado'}, axis=1)
df_empleados['id_empleado'] = df_empleados['id_empleado'] + 1
df_empleados

,id_empleado,nombre_empleado,puesto,id_tienda
0,1,Marta Vega,Cajero,1
1,2,José Martínez,Reponedor,1
2,3,Olga Hernández,Encargado de sección,1
3,4,Rafael Herrera,Gerente,1
4,5,Marta Serrano,Personal de limpieza,1
...,...,...,...,...
195,196,Gonzalo Rubio,Auxiliar de caja,10
196,197,Ricardo Medina,Encargado de inventario,10
197,198,Carmen Martínez,Responsable de compras,10
198,199,Jorge Ramos,Promotor de ventas,10


In [23]:
insertar_dataframe(df_empleados, "empleados")

200

categorias

* id_categoria: valores enteros consecutivos (1, 2, 3...).
* nombre_categoria: selección de categorías (p. ej. “Lácteos”, “Carnes”, “Frutas”, “Verduras”, “Bebidas”, “Snacks”).

En total 10 categorías

In [24]:
categorias = ['Lácteos', 'Carnes', 'Frutas', 'Verduras', 'Bebidas', 'Snacks', 'Hogar', 'Limpieza', 'Perfumería', 'Jardín']

df_categorias = pd.DataFrame(categorias, columns=['nombre_categoria'])

# generar columna id_categoria
df_categorias = df_categorias.reset_index().rename({'index': 'id_categoria'}, axis=1)
df_categorias['id_categoria'] = df_categorias['id_categoria'] + 1
df_categorias

,id_categoria,nombre_categoria
0,1,Lácteos
1,2,Carnes
2,3,Frutas
3,4,Verduras
4,5,Bebidas
5,6,Snacks
6,7,Hogar
7,8,Limpieza
8,9,Perfumería
9,10,Jardín


In [25]:
insertar_dataframe(df_categorias, 'categorias')

10


productos

* id_producto: valores enteros consecutivos (1, 2, 3...).
* nombre_producto: nombres como “Leche Entera”, “Manzana Roja”, “Refresco de Cola”, etc.
* precio: valores DECIMAL entre 0.50 y 50.00, por ejemplo.
* stock: valores INT entre 0 y 500 (o el rango que quieras).
* id_categoria: debe hacer referencia a las categorías que hayas definido (1, 2, 3, etc.).

En total 4 productos de cada categoría.

In [26]:
# generar columna id_producto
produ = ["Leche", "Yogur", "Queso", "Mantequilla", "Pollo", "Carne de res", "Cerdo", "Chuletas", "Manzana", "Banana", "Naranja", "Uva", "Zanahoria", "Tomate", "Lechuga", "Cebolla", "Agua", "Jugo de naranja", "Refresco", "Cerveza", "Papas fritas", "Chocolate", "Galletas", "Chicles", "Velas", "Platos desechables", "Bolsas de basura", "Pilas", "Detergente", "Jabón líquido", "Esponjas", "Desinfectante", "Champú", "Colonia", "Crema hidratante", "Desodorante", "Tierra para macetas", "Fertilizante", "Semillas", "Regadera"]
precios, lista_categorias, stock = list(), list(), list()
cantidad = len(produ)
for _ in range(cantidad):
    precios.append(random.uniform(0.50, 10.50))
    stock.append(random.randint(0, 500))
for categ in df_categorias['id_categoria']:
    for i in range(4):
        lista_categorias.append(categ)
lista_categorias        
df_productos = pd.DataFrame(zip(produ, precios, stock, lista_categorias), columns=['nombre_producto', 'precio', 'stock', 'id_categoria'])    
df_productos = df_productos.reset_index().rename({'index': 'id_producto'}, axis=1)
df_productos['id_producto'] = df_productos['id_producto'] + 1
df_productos

,id_producto,nombre_producto,precio,stock,id_categoria
0,1,Leche,9.197054,312,1
1,2,Yogur,8.089542,275,1
2,3,Queso,7.029539,238,1
3,4,Mantequilla,7.623484,116,1
4,5,Pollo,6.279399,179,2
5,6,Carne de res,9.546413,138,2
6,7,Cerdo,8.583987,372,2
7,8,Chuletas,8.469326,159,2
8,9,Manzana,7.970799,494,3
9,10,Banana,6.093617,202,3


In [27]:
insertar_dataframe(df_productos, 'productos')

40

clientes

* id_cliente: valores enteros consecutivos (1, 2, 3...).
* nombre_cliente: nombres y apellidos ficticios (p. ej. “Carlos López”, “María Gil”).
* email: podrías generar correos ficticios (p. ej. “carlos.lopez@test.com”).
* telefono: número de 9 o 10 dígitos (dependiendo del país).
* direccion: calles y números ficticios (p. ej. “Av. Siempre Viva 742”).

En total: 2000.


In [28]:
# generar columna id_cliente
nombre_cliente, apellido_cliente, email, postal, cliente_completo = list(), list(), list(), list(), list()
calles = ("Av. Libertad", "Calle Mayor", "Paseo de la Reforma", "Calle Real", "Av. de la Constitución", "Calle del Sol", "Calle de la Luna", "Calle los Pinos", "Calle de la Paz", "Calle Olivo", "Calle Cedro", "Calle Nogal", "Av. de las Flores", "Paseo del Prado", "Calle del Río", "Av. Central", "Calle Primavera", "Calle Otoño", "Calle Invierno", "Calle Verano", "Calle Granada", "Calle Sevilla", "Calle Córdoba", "Calle Málaga", "Calle Valencia", "Calle Zaragoza", "Calle Alicante", "Av. de los Ángeles", "Calle del Carmen", "Calle San Miguel", "Calle San Juan", "Calle del Pilar", "Calle Santa Teresa", "Calle San Francisco", "Calle de la Sierra", "Calle del Mar", "Av. del Norte", "Av. del Sur", "Calle del Este", "Calle del Oeste", "Calle Diamante", "Calle Esmeralda", "Calle Rubí", "Calle Zafiro", "Calle Ámbar", "Calle Turquesa", "Calle Topacio", "Calle Perla", "Calle Horizonte", "Calle Amanecer")
for _ in range(2000):
    nom = random.choice(nombres)
    nombre_cliente.append(nom)
    ape = random.choice(apellidos)
    apellido_cliente.append(ape)
    email.append((f'{nom}.{ape}@suemail.com'))
    postal.append(random.randint(11000, 11900))
    
df_clientes = pd.DataFrame(zip(nombre_cliente, apellido_cliente, email, postal), columns=['first_name', 'last_name', 'email', 'codigo_postal']) 
df_clientes = df_clientes.reset_index().rename({'index': 'id_cliente'}, axis=1)
df_clientes['id_cliente'] = df_clientes['id_cliente'] + 1
df_clientes

,id_cliente,first_name,last_name,email,codigo_postal
0,1,Enrique,Reyes,Enrique.Reyes@suemail.com,11327
1,2,Rafael,Fuentes,Rafael.Fuentes@suemail.com,11684
2,3,Gonzalo,Muñoz,Gonzalo.Muñoz@suemail.com,11574
3,4,Alberto,Campos,Alberto.Campos@suemail.com,11642
4,5,Alberto,Díaz,Alberto.Díaz@suemail.com,11240
...,...,...,...,...,...
1995,1996,Carlos,Campos,Carlos.Campos@suemail.com,11600
1996,1997,Paula,Rubio,Paula.Rubio@suemail.com,11263
1997,1998,Carlos,Medina,Carlos.Medina@suemail.com,11545
1998,1999,Antonio,Gutiérrez,Antonio.Gutiérrez@suemail.com,11163


In [29]:
insertar_dataframe(df_clientes, 'clientes')

2000

ordenes

* id_orden: valores enteros consecutivos (1, 2, 3...).
* id_cliente: haz referencia a los IDs existentes de la tabla clientes.
* id_empleado: haz referencia a los IDs existentes de la tabla empleados.
* fecha_orden: genera fechas aleatorias o secuenciales (p. ej. entre ‘2024-01-01’ y ‘2025-01-01’).
* metodo_pago: escoge entre {‘Tarjeta’, ‘Efectivo’}.

En total: 10000.


In [30]:
# generar columna id_orden
cliente, emplea, fecha, metodo = list(), list(), list(), list()
fechas = [datetime(year=2024, month=1, day=1) + timedelta(days=numero) for numero in range(395)]
for _ in range(10000):
    cliente.append(random.choice(df_clientes['id_cliente']))
    emplea.append(random.choice(df_empleados['id_empleado']))
    fecha.append(random.choice(fechas))
    metodo.append(random.choices(['Tarjeta', 'Efectivo'], weights=[0.65, 0.35])[0])

fecha.sort()
df_ordenes = pd.DataFrame(zip(cliente, emplea, fecha, metodo), columns=['id_cliente', 'id_empleado', 'fecha_orden', 'metodo_pago'])
df_ordenes.sort_values('fecha_orden', inplace=True)
df_ordenes = df_ordenes.reset_index().rename({'index': 'id_orden'}, axis=1)
df_ordenes['id_orden'] = df_ordenes['id_orden'] + 1
df_ordenes.sort_values('id_orden', inplace=True)
df_ordenes.head(50)

,id_orden,id_cliente,id_empleado,fecha_orden,metodo_pago
0,1,516,31,2024-01-01,Tarjeta
12,2,296,52,2024-01-01,Efectivo
16,3,327,69,2024-01-01,Tarjeta
14,4,1545,168,2024-01-01,Efectivo
15,5,1899,160,2024-01-01,Tarjeta
20,6,1255,178,2024-01-01,Tarjeta
17,7,1428,1,2024-01-01,Tarjeta
18,8,918,17,2024-01-01,Tarjeta
19,9,63,85,2024-01-01,Tarjeta
11,10,1827,104,2024-01-01,Tarjeta


In [31]:
insertar_dataframe(df_ordenes, 'ordenes')

10000

detalle_orden

* id_detalle: valores enteros consecutivos (1, 2, 3...).
* id_orden: referencia al ID de alguna orden válida.
* id_producto: referencia al ID de algún producto válido.
* cantidad: valores entre 1 y 20, por ejemplo.
* precio_unitario: usar el mismo precio que está en la tabla productos o uno ligeramente distinto si quieres simular ofertas.
* descuento: valores DECIMAL bajos (p. ej. 0.00, 1.00, 2.50) o NULL.

En total: 30000.

In [32]:
# generar columna id_detalle
ordenes, producto, cant, valor, descuento = list(), list(), list(), list(), list()
pre = pd.DataFrame(columns=['nombre_producto', 'precio', 'stock', 'id_categoria'])
for orden in df_ordenes['id_orden']:
    ordenes.append(orden)
    pre = ([df_productos[df_productos['id_producto']==random.choice(df_productos['id_producto'])]])
    producto.append(int(pre['id_producto']))
    cant.append(random.randint(1, 20))
    valor.append(pre('price'))  
    descuento.append(random.uniform(0, 0.25))   
df_detalle_orden = pd.DataFrame(zip[ordenes, producto, cant, valor, descuento], columns=['id_orden', 'id_producto', 'cantidad', 'precio_unitario', 'descuento'])
df_detalle_orden = df_detalle_orden.reset_index().rename({'index': 'id_detalle'}, axis=1)
df_detalle_orden['id_detalle'] = df_detalle_orden['id_detalle'] + 1
df_detalle_orden

TypeError: list indices must be integers or slices, not str

In [75]:
pre = list()
prod = random.choice(df_productos['id_producto'])
pre.append([prod, df_productos[df_productos['id_producto']==prod]['precio']])


In [38]:
pre = pd.DataFrame(columns=['nombre_producto', 'precio', 'stock', 'id_categoria'])
pre = ([df_productos[df_productos['id_producto']==random.choice(df_productos['id_producto'])]])
pre

[    id_producto nombre_producto    precio  stock  id_categoria
 20           21    Papas fritas  3.731691    127             6]

In [ ]:
insertar_dataframe(df_detalle_orden, 'detalle_orden')

### 3. Consultas SQL (34%)

1. Listado de órdenes con detalles de cliente y empleado
* Muestra el ID de la orden, la fecha, el nombre del cliente, el nombre del empleado que atendió la compra y el método de pago.
* Utiliza un JOIN entre las tablas ordenes, clientes y empleados.

2. Productos con stock bajo
* Filtra aquellos productos cuyo stock sea menor a 10.
* Muestra nombre del producto, categoría y stock.


3. Ventas totales por categoría
* Muestra el nombre de la categoría y la suma total de las ventas (ej.: multiplicando cantidad * precio_unitario) para cada categoría.
* Realiza el JOIN con detalle_orden, productos y categorias.
* Utiliza agrupación (GROUP BY).

4. Clientes con mayores gastos acumulados
* Muestra el nombre del cliente y el monto total que ha gastado (suma de todas sus órdenes).
* Asegúrate de tener en cuenta posibles descuentos (descuento) si se ha definido. Por ejemplo, la fórmula podría ser (cantidad * precio_unitario) - descuento.
* Ordena el resultado de mayor a menor gasto acumulado.

5. Empleados y número de órdenes gestionadas
* Muestra el nombre del empleado, el puesto y la cantidad de órdenes que ha gestionado.
* Utiliza GROUP BY y COUNT.

6. Ordenes filtradas por fecha y tienda
* Muestra todas las órdenes que se realizaron en un rango de fechas determinado (ej.: del 1 de enero de 2025 al 31 de enero de 2025) y en una tienda específica.
* Incluye datos de la tienda y del cliente.

7. Ranking de productos más vendidos en cada tienda
* Para cada tienda, muestra los 3 productos más vendidos (en términos de cantidad total).
* Tendrás que unir tiendas, empleados, ordenes y detalle_orden, además de productos.
* Usa GROUP BY y ordena por la cantidad sumada (y opcionalmente, un LIMIT 3).


Opcional: añadir alguna consulta con subconsultas o algo que no se abarque en las anteriores consulta.